<a href="https://colab.research.google.com/github/chakri9m/Chakri/blob/main/lerobot/training-deffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 x 🦾: Training pusht with LeRobot Notebook

Welcome to the **LeRobot pusht training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `pusht` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `diffusion-pusht` policy for 20,000 steps typically takes **about 5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [3]:
!git clone https://github.com/huggingface.co/lerobot/pusht.git
!cd pusht
!cd lerobot && pip install -e .


Cloning into 'pusht'...
remote: Not Found
fatal: repository 'https://github.com/huggingface.co/lerobot/pusht.git/' not found
/bin/bash: line 1: cd: pusht: No such file or directory
/bin/bash: line 1: cd: lerobot: No such file or directory


In [4]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 31222, done.
remote: Total 31222 (delta 0), reused 0 (delta 0), pack-reused 31222 (from 1)
Receiving objects: 100% (31222/31222), 162.22 MiB | 15.16 MiB/s, done.
Resolving deltas: 100% (20003/20003), done.
Filtering content: 100% (45/45), 69.04 MiB | 21.41 MiB/s, done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.1                 |       h166bdaf_1          69 KB  conda-forge
    ca-certificates-2025.7.14  |       hbd8a1cb_0         152 KB  conda-forge
    cairo-1.18.4      

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [5]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chakravardhan999 (chakravardhan999-sheffield-hallam-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Install pusht dependencies

In [6]:
!pip install -e lerobot


Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.2.0-0.editable-py3-none-any.whl size=17057 sha256=3533b53a857872355916df614f2f7f7affe27171837e76abf485dc42851f0ca2
  Stored in directory: /tmp/pip-ephem-wheel-cache-hk3em5t_/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.2.0
    Uninstalling lerobot-0.2.0:
      Successfully uninstalled lerobot-0.2.0


In [7]:
!cd lerobot && pip install -e


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-e option requires 1 argument


In [8]:
!ls /content/lerobot


benchmarks	    docker    LICENSE	   media	   src
CODE_OF_CONDUCT.md  docs      Makefile	   pyproject.toml  tests
CONTRIBUTING.md     examples  MANIFEST.in  README.md


In [9]:
!ls /content/lerobot/examples


1_load_lerobot_dataset.py	 4_train_policy_with_script.md
2_evaluate_pretrained_policy.py  backward_compatibility
3_train_policy.py		 lekiwi


In [10]:
!find /content/lerobot -name train.py


/content/lerobot/src/lerobot/scripts/train.py
/content/lerobot/src/lerobot/configs/train.py


In [11]:
!pip install datasets



In [12]:
!python /content/lerobot/src/lerobot/scripts/train.py\
    --output_dir=outputs/train/diffusion_pusht \
    --policy.type=diffusion \
    --dataset.repo_id=lerobot/pusht \
    --seed=100000 \
    --env.type=pusht \
    --batch_size=64 \
    --steps=200000 \
    --eval_freq=25000 \
    --save_freq=25000 \
    --wandb.enable=true \
    --policy.repo_id=local



INFO 2025-08-01 04:25:43 ils/utils.py:48 Cuda backend detected, using cuda.
WARNING 2025-08-01 04:25:43 /policies.py:79 Device 'None' is not available. Switching to 'cuda'.
INFO 2025-08-01 04:25:43 ts/train.py:111 {'batch_size': 64,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'Colo

In [13]:
!pip install gym-pusht


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 148.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 52.6 MB/s eta 0:00:00


In [14]:
!pip install pymunk==6.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 81.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pymunk: filename=pymunk-6.2.1-cp311-cp311-linux_x86_64.whl size=284448 sha256=f1d8ebba73d8b776762cc6ed14517fdc06b406d7555a3d25a7580b31e4805b2f
  Stored in directory: /root/.cache/pip/wheels/16/23/6d/2e713157384d80a5b142e50188d3707f1146d0e3231a4f2157
Successfully built pymunk
  Attempting uninstall: pymunk
    Found existing installation: pymunk 7.1.0
    Uninstalling pymunk-7.1.0:
      Successfully uninstalled pymunk-7.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym-pusht 0.1.5 requires pymunk>=6.6.0, but you have pymunk 6.2.1 which is incompatible.


In [15]:
!python /content/lerobot/examples/2_evaluate_pretrained_policy.py\
    --policy.path=outputs/train/diffusion_pusht/checkpoints/175000/pretrained_model \
    --env.type=pusht \
    --eval.n_episodes=500 \
    --eval.batch_size=50


config.json: 1.51kB [00:00, 5.90MB/s]
model.safetensors: 100% 1.05G/1.05G [00:13<00:00, 76.4MB/s]
{'observation.image': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 96, 96)), 'observation.state': PolicyFeature(type=<FeatureType.STATE: 'STATE'>, shape=(2,))}
Dict('agent_pos': Box(0.0, 512.0, (2,), float64), 'pixels': Box(0, 255, (96, 96, 3), uint8))
{'action': PolicyFeature(type=<FeatureType.ACTION: 'ACTION'>, shape=(2,))}
Box(0.0, 512.0, (2,), float32)
step=0 reward=np.float64(0.0) terminated=False
step=1 reward=np.float64(0.0) terminated=False
step=2 reward=np.float64(0.0) terminated=False
step=3 reward=np.float64(0.0) terminated=False
step=4 reward=np.float64(0.0) terminated=False
step=5 reward=np.float64(0.0) terminated=False
step=6 reward=np.float64(0.0) terminated=False
step=7 reward=np.float64(0.0) terminated=False
step=8 reward=np.float64(0.0) terminated=False
step=9 reward=np.float64(0.0) terminated=False
step=10 reward=np.float64(0.0) terminated=False
step=11 r

In [16]:

from IPython.display import Video

Video("/content/outputs/eval/example_pusht_diffusion/rollout.mp4", embed=True, width=600)


code for mutiple locations of push t


In [17]:
# Install required system dependencies
!apt-get update && apt-get install -y ffmpeg xvfb


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,775 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,207 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages 

In [18]:
!pip install pymunk==6.6.0
!pip install "gymnasium>=0.29.1,<1.0.0"
!pip install git+https://github.com/huggingface/gym-pusht.git
!pip install git+https://github.com/huggingface/lerobot.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
  Attempting uninstall: pymunk
    Found existing installation: pymunk 6.2.1
    Uninstalling pymunk-6.2.1:
      Successfully uninstalled pymunk-6.2.1
  Cloning https://github.com/huggingface/gym-pusht.git to /tmp/pip-req-build-8d2iwx2e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/gym-pusht.git /tmp/pip-req-build-8d2iwx2e
  Resolved https://github.com/huggingface/gym-pusht.git to commit 5e2489be9ff99ed9cd47b6c653dda3b7aa844d24
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/lerobot.git to /tmp/pip-req-build-xa0oi11p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/lerobot.git /tmp/pip-req-build-xa0oi11p
  Resolved https://github.com/huggingface/lerobot.git to commit 945e1ff2669bb7b31cb7fe6033fe9679767c2442


In [19]:
import os
import subprocess

# Start a dummy display so rendering (used by RecordVideo) works
subprocess.Popen(["Xvfb", ":1", "-screen", "0", "1280x720x24"])
os.environ["DISPLAY"] = ":1"


In [20]:
# 🔧 Imports
import os
import shutil
import datetime
from pathlib import Path

import torch
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import gym_pusht
from lerobot.policies.diffusion.modeling_diffusion import DiffusionPolicy

# 🚀 Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 🎯 Load pretrained policy
policy = DiffusionPolicy.from_pretrained("lerobot/diffusion_pusht")
policy.to(device)
policy.eval()
policy.reset()

# 🗂️ Output directory setup
output_dir = Path("outputs/eval/multiple_location")
shutil.rmtree(output_dir, ignore_errors=True)
output_dir.mkdir(parents=True, exist_ok=True)

# 📍 5 different object positions
locations = [
    [0.0,  0.0,  0.0, 0.0, 1.0],
    [0.1, -0.1,  0.0, 0.0, 1.0],
    [-0.1, 0.05, 0.0, 0.0, 1.0],
    [0.05, 0.2,  0.0, 0.0, 1.0],
    [-0.2, -0.1, 0.0, 0.0, 1.0],
]

# 🧪 Create base PushT env
base_env = gym.make(
    "gym_pusht/PushT-v0",
    obs_type="pixels_agent_pos",
    render_mode="rgb_array",
    max_episode_steps=300,
)

# 🌀 Run each rollout
for i, loc in enumerate(locations, 1):
    print(f"\n▶️ Rollout {i} | Object position: {loc}")

    # Reset base env to desired object position
    base_env.reset(options={"reset_to_state": loc})

    # Timestamped folder & prefix
    rollout_dir = output_dir / f"rollout_{i}"
    rollout_dir.mkdir(parents=True, exist_ok=True)
    time_prefix = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    name_prefix = f"pusht_loc_{i}_{time_prefix}"

    # Wrap for video recording
    env = RecordVideo(
        base_env,
        video_folder=str(rollout_dir),
        name_prefix=name_prefix,
        episode_trigger=lambda eid: True,
        disable_logger=True,
    )

    obs, _ = env.reset()
    done = False
    total_reward = 0
    step = 0

    # 🕹️ Step through till done
    while not done and step < 300:
        state = torch.tensor(obs["agent_pos"], dtype=torch.float32).unsqueeze(0).to(device)
        image = torch.tensor(obs["pixels"], dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).div(255).to(device)

        with torch.no_grad():
            action = policy.select_action({
                "observation.state": state,
                "observation.image": image,
            })

        obs, reward, terminated, truncated, _ = env.step(action.squeeze(0).cpu().numpy())
        done = terminated or truncated
        total_reward += reward
        step += 1

    env.close()  # ⚠️ Important to finalize video
    print(f"✅ Rollout {i} done - Steps: {step}, Reward: {total_reward:.2f}")


Using device: cuda


/usr/local/lib/python3.11/site-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



▶️ Rollout 1 | Object position: [0.0, 0.0, 0.0, 0.0, 1.0]


/usr/local/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/outputs/eval/multiple_location/rollout_1 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


✅ Rollout 1 done - Steps: 268, Reward: 109.41

▶️ Rollout 2 | Object position: [0.1, -0.1, 0.0, 0.0, 1.0]


...

✅ Rollout 3 done - Steps: 300, Reward: 166.88

▶️ Rollout 4 | Object position: [0.05, 0.2, 0.0, 0.0, 1.0]


/usr/local/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/outputs/eval/multiple_location/rollout_4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


✅ Rollout 4 done - Steps: 234, Reward: 114.29

▶️ Rollout 5 | Object position: [-0.2, -0.1, 0.0, 0.0, 1.0]


/usr/local/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /content/outputs/eval/multiple_location/rollout_5 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


✅ Rollout 5 done - Steps: 300, Reward: 168.83


In [70]:
from IPython.display import Video, display

# Change rollout_1 to 2, 3, ... to view other episodes
video_path = "/content/outputs/eval/multiple_location/rollout_1/pusht_loc_1_20250801_043026-episode-0.mp4"
display(Video(video_path, embed=True))

In [71]:
from IPython.display import Video, display

# Change rollout_1 to 2, 3, ... to view other episodes
video_path = "/content/outputs/eval/multiple_location/rollout_2/pusht_loc_2_20250801_043133-episode-0.mp4"
display(Video(video_path, embed=True))

In [72]:
from IPython.display import Video, display

# Change rollout_1 to 2, 3, ... to view other episodes
video_path = "/content/outputs/eval/multiple_location/rollout_3/pusht_loc_3_20250801_043246-episode-0.mp4"
display(Video(video_path, embed=True))

In [73]:
from IPython.display import Video, display

# Change rollout_1 to 2, 3, ... to view other episodes
video_path = "/content/outputs/eval/multiple_location/rollout_4/pusht_loc_4_20250801_043401-episode-0.mp4"
display(Video(video_path, embed=True))

In [74]:
from IPython.display import Video, display

# Change rollout_1 to 2, 3, ... to view other episodes
video_path = "/content/outputs/eval/multiple_location/rollout_5/pusht_loc_5_20250801_043458-episode-0.mp4"
display(Video(video_path, embed=True))

In [26]:
!python /content/lerobot/src/lerobot/scripts/eval.py \
    --policy.path=outputs/train/diffusion_pusht/checkpoints/175000/pretrained_model \
    --output_dir=outputs/eval/diffusion_pusht/175000 \
    --env.type=pusht \
    --eval.n_episodes=500 \
    --eval.batch_size=50


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/draccus/parsers/decoding.py", line 159, in decode_dataclass
    instance = origin(**init_args)
               ^^^^^^^^^^^^^^^^^^^
  File "<string>", line 9, in __init__
  File "/usr/local/lib/python3.11/site-packages/lerobot/configs/eval.py", line 43, in __post_init__
    self.policy = PreTrainedConfig.from_pretrained(policy_path, cli_overrides=cli_overrides)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/lerobot/configs/policies.py", line 172, in from_pretrained
    config_file = hf_hub_download(
                  ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/utils/_validators.py", line 106, in _inner_fn
    validate_repo_id(arg_value)
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/utils/_validators.py", line 154, in validate_repo_id
    raise HFValidationErro

In [27]:
!python /content/lerobot/src/lerobot/scripts/eval.py \
    --policy.path=outputs/train/diffusion_pusht/checkpoints/175000/pretrained_model \
    --output_dir=outputs/eval/diffusion_pusht/175000 \
    --env.type=pusht \
    --eval.n_episodes=500 \
    --eval.batch_size=50 \
    --device=cuda \
    --use_amp=false

usage: eval.py [-h] [--config_path str] [--env str]
               [--env.type {aloha,pusht,xarm,gym_manipulator,hil}]
               [--env.obs_type str] [--env.render_mode str]
               [--env.visualization_width str]
               [--env.visualization_height str] [--robot str]
               [--env.robot.type str] [--teleop str] [--env.teleop.type str]
               [--env.task str] [--env.fps str] [--env.features str]
               [--env.features_map str] [--env.name str]
               [--env.use_viewer str] [--env.gripper_penalty str]
               [--env.use_gamepad str] [--env.state_dim str]
               [--env.action_dim str] [--env.episode_length str]
               [--video_record str] [--env.video_record.enabled str]
               [--env.video_record.record_dir str]
               [--env.video_record.trajectory_name str]
               [--env.reward_classifier_pretrained_path str]
               [--robot_config str] [--env.robot_config.type str]
              

In [28]:
%cd /content/lerobot
!pip install -e .

/content/lerobot
Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.2.0-0.editable-py3-none-any.whl size=17057 sha256=da7b2dd6c42aac0ed62d081382f11331586e3b24d32a4b06ec30682bee68e3e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-_q2wla_v/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.2.0
    Uninstalling lerobot-0.2.0:
      Successfully uninstalled lerobot-0.2.0


In [29]:
!ls -R /content/lerobot | head -n 100



/content/lerobot:
benchmarks
CODE_OF_CONDUCT.md
CONTRIBUTING.md
docker
docs
examples
LICENSE
Makefile
MANIFEST.in
media
pyproject.toml
README.md
src
tests

/content/lerobot/benchmarks:
video

/content/lerobot/benchmarks/video:
capture_camera_feed.py
README.md
run_video_benchmark.py

/content/lerobot/docker:
Dockerfile.internal
Dockerfile.user

/content/lerobot/docs:
README.md
source

/content/lerobot/docs/source:
async.mdx
backwardcomp.mdx
cameras.mdx
contributing.md
hilserl.mdx
hilserl_sim.mdx
hope_jr.mdx
il_robots.mdx
il_sim.mdx
index.mdx
installation.mdx
integrate_hardware.mdx
koch.mdx
lekiwi.mdx
notebooks.mdx
smolvla.mdx
so100.mdx
so101.mdx
_toctree.yml

/content/lerobot/examples:
1_load_lerobot_dataset.py
2_evaluate_pretrained_policy.py
3_train_policy.py
4_train_policy_with_script.md
backward_compatibility
lekiwi

/content/lerobot/examples/backward_compatibility:
replay.py

/content/lerobot/examples/lekiwi:
evaluate.py
record.py
replay.py
teleoperate.py

/content/lerobot/media:
gy

part 2

In [30]:
!ls /content/lerobot/lerobot/scripts

ls: cannot access '/content/lerobot/lerobot/scripts': No such file or directory


In [31]:
!python /content/lerobot/lerobot/scripts/eval.py  --help


python: can't open file '/content/lerobot/lerobot/scripts/eval.py': [Errno 2] No such file or directory


In [32]:
from huggingface_hub import notebook_login
notebook_login()



In [33]:
import json
import os

info = {
    "codebase_version": "0.0.1",
    "features": {
        "image": {
            "dtype": "uint8",
            "shape": [128, 128, 3]
        },
        "action": {
            "dtype": "float32",
            "shape": [7]
        }
    }
}

os.makedirs("/content/move_to_box/meta", exist_ok=True)

with open("/content/move_to_box/meta/info.json", "w") as f:
    json.dump(info, f, indent=2)

print("✅ Fixed info.json written.")


✅ Fixed info.json written.


In [34]:
!git clone https://huggingface.co/Frontier-Machines/move_to_box


Cloning into 'move_to_box'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 150 (delta 1), reused 0 (delta 0), pack-reused 3 (from 1)
Receiving objects: 100% (150/150), 39.29 MiB | 17.30 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Filtering content: 100% (47/47), 1018.23 MiB | 19.51 MiB/s, done.


In [35]:
!cd move_to_box && ls -l



total 1041796
-rw-r--r-- 1 root root       1209 Aug  1 04:36 config.json
drwxr-xr-x 2 root root       4096 Aug  1 04:36 episodes
drwxr-xr-x 2 root root       4096 Aug  1 04:36 meta_data
-rw-r--r-- 1 root root 1066761964 Aug  1 04:37 model.safetensors
-rw-r--r-- 1 root root        385 Aug  1 04:36 README.md
drwxr-xr-x 2 root root       4096 Aug  1 04:36 train
drwxr-xr-x 2 root root      12288 Aug  1 04:36 videos


In [36]:
!head -40 move_to_box/README.md


---
library_name: lerobot
tags:
- diffusion-policy
- model_hub_mixin
- pytorch_model_hub_mixin
- robotics
---

This model has been pushed to the Hub using the [PytorchModelHubMixin](https://huggingface.co/docs/huggingface_hub/package_reference/mixins#huggingface_hub.PyTorchModelHubMixin) integration:
- Library: https://github.com/huggingface/lerobot
- Docs: [More Information Needed]

In [37]:
# Change this to any of the supported ones from HuggingFace
!pretrained_policy_path = "lerobot/diffusion_open_drawer"

!policy = DiffusionPolicy.from_pretrained(pretrained_policy_path)


/bin/bash: line 1: pretrained_policy_path: command not found
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `policy = DiffusionPolicy.from_pretrained(pretrained_policy_path)'


In [38]:
!pip install lerobot


In [39]:
ls -l /content/movetobox.py


ls: cannot access '/content/movetobox.py': No such file or directory


In [40]:
import lerobot
import pkgutil

print("lerobot submodules:")
for mod in pkgutil.iter_modules(lerobot.__path__):
    print(mod.name)


lerobot submodules:


In [41]:
!python /content/drive/MyDrive/movetobox.py


{'observation.image': PolicyFeature(type=<FeatureType.VISUAL: 'VISUAL'>, shape=(3, 96, 96)), 'observation.state': PolicyFeature(type=<FeatureType.STATE: 'STATE'>, shape=(2,))}
Dict('agent_pos': Box(0.0, 512.0, (2,), float64), 'pixels': Box(0, 255, (96, 96, 3), uint8))
{'action': PolicyFeature(type=<FeatureType.ACTION: 'ACTION'>, shape=(2,))}
Box(0.0, 512.0, (2,), float32)
step=0 reward=np.float64(0.0) terminated=False
step=1 reward=np.float64(0.0) terminated=False
step=2 reward=np.float64(0.0) terminated=False
step=3 reward=np.float64(0.0) terminated=False
step=4 reward=np.float64(0.0) terminated=False
step=5 reward=np.float64(0.0) terminated=False
step=6 reward=np.float64(0.0) terminated=False
step=7 reward=np.float64(0.0) terminated=False
step=8 reward=np.float64(0.0) terminated=False
step=9 reward=np.float64(0.0) terminated=False
step=10 reward=np.float64(0.0) terminated=False
step=11 reward=np.float64(0.0) terminated=False
step=12 reward=np.float64(0.0) terminated=False
step=13 rew

In [69]:
from IPython.display import Video, display

# Change rollout_1 to 2, 3, ... to view other episodes
video_path = "/content/lerobot/outputs/eval/example_pusht_diffusion/rollout.mp4"
display(Video(video_path, embed=True))

In [43]:
!ls /content/lerobot/src



lerobot  lerobot.egg-info


In [44]:
import sys
sys.path.append('/content/lerobot/src')


WHy?

In [45]:
!python -m lerobot.datasets.v2.convert_dataset_v1_to_v2 \
    --repo-id Frontier-Machines/move_to_box \
    --single-task "Pick up the cube and place it into the box." \
    --robot koch_follower



Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/lerobot/src/lerobot/datasets/v2/convert_dataset_v1_to_v2.py", line 687, in <module>
    main()
  File "/content/lerobot/src/lerobot/datasets/v2/convert_dataset_v1_to_v2.py", line 679, in main
    robot_config = make_robot_config(args.robot)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/lerobot/src/lerobot/datasets/v2/convert_dataset_v1_to_v2.py", line 607, in make_robot_config
    return KochFollowerConfig(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: KochFollowerConfig.__init__() missing 1 required positional argument: 'port'


In [46]:
!pip install gymnasium imageio torch gym_pusht


In [47]:
!pip install gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 24.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827726 sha256=f4eb9317e1adef6d5f3339c0563c979fe55526b3b92ed33bd9333f657057c212
  Stored in directory: /root/.cache/pip/wheels/1c/77/9e/9af5470201a0b0543937933ee99ba884cd237d2faefe8f4d37
Successfully built gym


In [48]:
!python "/content/movetobox.py"


python: can't open file '/content/movetobox.py': [Errno 2] No such file or directory


In [49]:
!python -m lerobot.scripts.visualize_dataset \
    --repo-id=lerobot/pusht \
    --episode-index=0 \
    --output-dir=/content/lerobot/outputs/visualized_episode \
    --save=1 \
    --mode=rgb



Fetching 418 files:  50% 208/418 [00:00<00:00, 983.94it/s]
videos/chunk-000/observation.image/episo(…):   0% 0.00/26.5k [00:00<?, ?B/s]

videos/chunk-000/observation.image/episo(…):   0% 0.00/23.2k [00:00<?, ?B/s]


videos/chunk-000/observation.image/episo(…):   0% 0.00/35.7k [00:00<?, ?B/s]



videos/chunk-000/observation.image/episo(…):   0% 0.00/34.4k [00:00<?, ?B/s]




videos/chunk-000/observation.image/episo(…):   0% 0.00/22.7k [00:00<?, ?B/s]





videos/chunk-000/observation.image/episo(…):   0% 0.00/33.8k [00:00<?, ?B/s]






videos/chunk-000/observation.image/episo(…):   0% 0.00/42.1k [00:00<?, ?B/s]







videos/chunk-000/observation.image/episo(…):   0% 0.00/39.2k [00:00<?, ?B/s]

videos/chunk-000/observation.image/episo(…): 100% 23.2k/23.2k [00:01<00:00, 17.7kB/s]




videos/chunk-000/observation.image/episo(…): 100% 34.4k/34.4k [00:01<00:00, 26.1kB/s]








videos/chunk-000/observation.image/episo(…): 100% 39.2k/39.2k [00:01<00:00, 29.6kB/s]



videos/chunk-000/observ

In [50]:
!python -m lerobot.scripts.visualize_dataset \
    --repo-id lerobot/pusht \
    --root ./my_local_data_dir \
    --local-files-only 1 \
    --episode-index 0

usage: visualize_dataset.py [-h] --repo-id REPO_ID --episode-index
                            EPISODE_INDEX [--root ROOT]
                            [--output-dir OUTPUT_DIR]
                            [--batch-size BATCH_SIZE]
                            [--num-workers NUM_WORKERS] [--mode MODE]
                            [--web-port WEB_PORT] [--ws-port WS_PORT]
                            [--save SAVE] [--tolerance-s TOLERANCE_S]
visualize_dataset.py: error: unrecognized arguments: --local-files-only 1


## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [51]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
object address  : 0x7da44d28c940
object refcount : 2
object type     : 0x5c4c31b853e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()

In [53]:
!git clone https://huggingface.co/Frontier-Machines/move_to_box

fatal: destination path 'move_to_box' already exists and is not an empty directory.


In [54]:
cd move_to_box


/content/lerobot/move_to_box


In [55]:
ls


config.json  meta_data/         README.md  videos/
episodes/    model.safetensors  train/


In [56]:
ls train/


data-00000-of-00001.arrow  dataset_info.json  state.json


In [57]:
cat config.json


{
  "beta_end": 0.02,
  "beta_schedule": "squaredcos_cap_v2",
  "beta_start": 0.0001,
  "clip_sample": true,
  "clip_sample_range": 1.0,
  "crop_is_random": true,
  "crop_shape": [
    220,
    300
  ],
  "diffusion_step_embed_dim": 128,
  "do_mask_loss_for_padding": false,
  "down_dims": [
    512,
    1024,
    2048
  ],
  "horizon": 16,
  "input_normalization_modes": {
    "observation.images.left": "mean_std",
    "observation.images.right": "mean_std",
    "observation.state": "mean_std"
  },
  "input_shapes": {
    "observation.images.left": [
      3,
      240,
      320
    ],
    "observation.images.right": [
      3,
      240,
      320
    ],
    "observation.state": [
      7
    ]
  },
  "kernel_size": 5,
  "n_action_steps": 16,
  "n_groups": 8,
  "n_obs_steps": 2,
  "noise_scheduler_type": "DDIM",
  "num_inference_steps": 10,
  "num_train_timesteps": 100,
  "output_normalization_modes": {
    "action": "min_max"
  },
  "output_shapes": {
    "action": [
      7
    ]
  

In [59]:
ls -R /content/move_to_box/train


ls: cannot access '/content/move_to_box/train': No such file or directory


In [60]:
ls -R /content/move_to_box


/content/move_to_box:
meta/

/content/move_to_box/meta:
info.json


In [61]:
!git clone https://huggingface.co/Frontier-Machines/move_to_duct_tape

Cloning into 'move_to_duct_tape'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (9/9), 3.20 KiB | 820.00 KiB/s, done.


In [63]:
cd move_to_duct_tape

/content/lerobot/move_to_box/move_to_duct_tape


In [64]:
ls

config.json  meta.json  model.safetensors  README.md


In [65]:
!head -40 /content/move_to_box/move_to_duct_tape/README.md


head: cannot open '/content/move_to_box/move_to_duct_tape/README.md' for reading: No such file or directory


In [66]:
!ls /content/lerobot

benchmarks	    docker    Makefile	   outputs	   tests
checkpoints	    docs      MANIFEST.in  pyproject.toml
CODE_OF_CONDUCT.md  examples  media	   README.md
CONTRIBUTING.md     LICENSE   move_to_box  src


In [67]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="lerobot/pusht",
    repo_type="dataset",
    local_dir="./my_local_data_dir",
    local_dir_use_symlinks=False
)


/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

meta/tasks.parquet:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

data/chunk-000/file-000.parquet:   0%|          | 0.00/674k [00:00<?, ?B/s]

videos/observation.image/chunk-000/file-(…):   0%|          | 0.00/6.89M [00:00<?, ?B/s]

...

info.json: 0.00B [00:00, ?B/s]

stats.json: 0.00B [00:00, ?B/s]

meta/episodes/chunk-000/file-000.parquet:   0%|          | 0.00/107k [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

'/content/lerobot/move_to_box/move_to_duct_tape/my_local_data_dir'

In [68]:
!python -m lerobot.scripts.visualize_dataset \
  --repo-id lerobot/pusht \
  --root ./my_local_data_dir \
  --episode-index 0 \
  --mode local



The dataset you requested (lerobot/pusht) is in 3.0 format.
While current version of LeRobot is backward-compatible with it, the version of your dataset still uses global
stats instead of per-episode stats. Update your dataset stats to the new format using this command:
```
python -m lerobot.datasets.v21.convert_dataset_v20_to_v21 --repo-id=lerobot/pusht
```

If you encounter a problem, contact LeRobot maintainers on [Discord](https://discord.com/invite/s3KuuzsPFb)
or open an [issue on GitHub](https://github.com/huggingface/lerobot/issues/new/choose).

Fetching 4 files:   0% 0/4 [00:00<?, ?it/s]
episodes.jsonl: 20.8kB [00:00, 51.0MB/s]

info.json: 2.72kB [00:00, 14.6MB/s]
Fetching 4 files:  25% 1/4 [00:00<00:00,  3.57it/s]
tasks.jsonl: 100% 79.0/79.0 [00:00<00:00, 406kB/s]

episodes_stats.jsonl: 333kB [00:00, 65.3MB/s]
Fetching 4 files: 100% 4/4 [00:00<00:00, 13.84it/s]
Fetching 418 files:   0% 1/418 [00:00<01:34,  4.41it/s]
data/chunk-000/episode_000003.parquet:   0% 0.00/11.2k [00:00

In [62]:
!pip install "lerobot[pusht]"
